## Usando $eq y $ne

Empezaremos a usar consultas mas complejas con los query operators, como son:

- $eq (equal)
- $ne (not equal)

### eq

Lo hemos usado de manera implicita por ejemplo, buscando los de cantidad igual a 20.

    db.inventory.find( {qty: 20} )

Si lo queremos usar de manera explicita, agregamos el operador:

    db.inventory.find( {qty: { $eq:20 }} )

Y retorna:

    _id: 2, item: { name: 'cd', code: '123' }, qty: 20, tags: [ 'B' ] 

### Busqueda dentro de un subdocumento.

Supongamso que queremos mostrar todas las coincidencias de *code* con 123. Y como puedes observar el atributo item en si es un subdocumento.

Aqui de manera implicita


    db.inventory.find(
        { "item.code": "123"  }
    )

Y aca de manera explicita

    db.inventory.find(
        { "item.code": { $eq:"123"} }
    )

### not equal

$ne solo se puedes usar de manera explicita. Aqui todos los documentos cuya cantidad no es igual a 20.

    db.inventory.find( {qty: { $ne:20 }} )

De igual forma se puede usar eq y ne para un update

#### Ejemplo

Realizar una actualizacion del incremento del precio para todos los documentos cuya cantidad no es igual a 20.


    db.inventory.updateMany(
        {qty : { $ne: 20}},
        { $inc: { qty:3}}
    )

#### Ejemplo

Agregar un nuevo tag llamado Z para todos los documentos cuyo nombre no es igual a *ij*.

    db.inventory.updateMany(
        { "item.name": {$ne: "ij"}},
        {
            $push: { tags: "z"}
        }
    )

## Usando $gt, $gte, $lt, $lte

Creando un nuevo folder :*112_operators*

Operadores:

- $gt (greater than - >)
- $gte (greater than or equal - >=)
- $lt (less than - <)
- $lte (less than or equal - <=)

### Usando gt y gte

Buscando los documentos que sean mayores que 20:

    db.inventory.find( { qty: { $gt: 20} } )

Y aqui mayores o iguales a 20:

    db.inventory.find( { qty: { $gte: 20} } )

### Usando lt and lte

Menor que 20

    db.inventory.find( { qty: { $lt: 20} } )

Menor o igual que 20

    db.inventory.find( { qty: { $lte: 20} } )

### Ejercicio

Buscar los documentos mayor o igual que 25 y menor o igual que 35

    db.inventory.find( { qty: { $gte: 25, $lte: 35} } )

### Ejercicio

Que cantidad sea mayor o igual que 50, y que el codigo no sea 090.

    db.inventory.find( 
        { qty: { $gte: 50},
        "item.code": { $ne: "090" } }
    )

## Ejemplos de $gt, $gte, $lt, $lte

Para el sensor 1, quitar las lecturas mayores que 2 o menores o iguales que 5

    db.iot.updateMany(
        { sensor: { $eq: "A001"} },
        { 
        $pull: {
        readings: { $gt: 2, $lte: 5 }
        }}
    )

Realizaremos nuevas consultas pero esta vez a la BD *sample_training*

La primera consulta sera todos los viajes que duren menos de 200 segundos, y contemolos:

    db.trips.find( { tripduration: { $lte: 200 }} ).count()

Y de los viajes hechos anteriormente, cuales de estos viajes fueron hechos por usuarios que tienen una suscripcion:

    db.trips.find( 
        { tripduration: { $lte: 200 }, usertype: { $eq: "Subscriber"} } ).count()


## Usando $regex 

El operador *regex* hace busquedas de patrones dentro del texto, con expresiones regulares. Crearemos el folder *14-regex*. EL dataser lo copiamos de:

https://gist.github.com/nicobytes/fbd8c63977217855ba8afd3e240651c9

### Ejercicio de repaso

Buscar dentro del subdocumento item los documentos cuyo codigo no es igual a *000*

    db.inventory.find({"item.code": { $ne: "000"}})


### Ejemplo

Buscar dentro del subdocumento item los documentos en cuya descripcion esta la palabra *line*

Como la busqueda se hace con expresiones regulares, se insertan con un doble backslash

    db.inventory.find({ "item.description": { $regex: /line/ } })

La busqueda anterior no muestra el documento 6, a pesar de que la palabra line esta dentro de la descripcion pero en mayusculas, ¿que hacer?

Hay una forma de decirle a la expresion regular no sea *sensitive case* y es agregando una *i* al final de la misma.

    db.inventory.find({ "item.description": { $regex: /line/i } })

Otro caso interesante, mostrar aquellos que solo al final tienen la palabra *line*, en este caso serian solo dos documentos. Y para hacerlo con el simbolo *$* asi tal como se muestra:

    db.inventory.find({ "item.description": { $regex: /line$/i } })

Y el resultado:

    {
        _id: 2,
        item: { name: 'cd', code: '123', description: 'First line\nSecond line' },
        qty: 20,
        tags: [ 'B' ]
    },
    {
        _id: 3,
        item: {
        name: 'ij',
        code: '456',
        description: 'Many spaces before     line'
        },
        qty: 25,
        tags: [ 'A', 'B' ]
    }




### Ejemplo. Buscar un texto que inicie en...

En este caso los que inician con la palabra *single*

Se hace con el caracter *^* SHIFT + Tecla

    db.inventory.find({ "item.description": { $regex: /^single/i } })

Y efectivamente retorna dos documentos.

### Saltos de lineas

Si realizamos la siguiente busqueda de todos los documentos que inician por *s* , podria ser:

    db.inventory.find({ "item.description": { $regex: /^single/i } })

Y mostraria dos:

    "Single line description."
    "Single LINE description"

¿Pero que pasaria con el documento que contiene la cadena de texto *'First line\nSecond line'*?

Este ultimo contiene un salto de linea, un nuevo parrafo, pero igualmente ¿si quisiera buscar todos los documentos que inicien con *s* aun asi con los saltos de linea?

Con el operados *m* de multilie, para textos o parrafos que tengan saltos de linea:

    db.inventory.find({ "item.description": { $regex: /^s/im } })

Como conclusion el operador de *Regex* funciona bien y es versatil, pero se puede quedar corto si se quiere implementar un buen buscador.

MongoDB tienen *MongdoDB Atlas Search* con features muy avanzados incluso en AI, entre otros, y es algo avanzado

## Projection

Conectemonos a mongoDB y realizemos una consulta sobre *sample_training*

    use("sample_training")
    
    db.trips.find( 
        { tripduration: { $lte: 200 }, usertype: { $eq: "Subscriber"} } )


Y me retorna todos los campos, sin embargo solo me interesa el que me interesa. Iremos a *Compass*. 

![](https://i.imgur.com/ob2fQ3e.png)

Como ves, en la opcion Project coloco los campos a visualizar, junto con un 1:

    { tripduration: 1}

El id lo mostrara siempre, a menos le indique lo contrario con un 0:

    { tripduration: 1, _id: 0}

Si queremso seguir agregando otros campos:

    { tripduration: 1, _id: 0, "start station location": 1}

### En los playgrouns:

    db.trips.find(
        // querry
        { tripduration: { $gte: 190, $lte: 200 }, usertype: { $eq: "Subscriber"} },

        //projection
        { tripduration: 1}
    )

190 <= tripduration <= 200. 

Y esta sera la salida:

    {
    "_id": {
      "$oid": "572bb8222b288919b68abf93"
    },
    "tripduration": 193
    },

## Operadores para Arrays

Usaremos dos datasets. QUe estaran en un mismo archivo. Crearemos una nueva carpeta *16-query-arrays*

### Primer Dataset

Haremos una consulta tipo projecion para mostrar los tags sobre los cuales vamos hacer consultas.

    db.inventory.find({}, {tags:1})

Y el resultado:

    {
    "_id": 1,
    "tags": [
      "school",
      "book",
      "bag",
      "headphone",
      "appliance"
    ]
  },

### Segundo Dataset

Es un poco mas complejo, pero igual no tanto.

### Operador in

Como ejemplo buscar todos los documentos donde el tag contenga "book", con el projection solo mostrara los tags

  use("platzi_store")
  db.inventory.find(
      { tags: { $in: ["book"] }},
      { tags:1})

Aqui lo que muestra:

![](https://i.imgur.com/UvFJSWA.png)



Realizemos una nueva busqueda donde el objeto no es un array, e igulmente funcionara:

    use("platzi_store")
    db.inventory.find(
        { qty: { $in: [20, 25] }},
        { qty:1})

Acabamos de buscar si la cantida es 20 0 25.

### Operador nin

Es el contrario de *in*. Por ejemplo los que no contienen ni book ni appliance.

    db.inventory.find(
        { tags: { $nin: ["book", "appliance"] }},
        { tags:1})

### Operadores exclusivos para Arrays

#### all

Busquemos todos los que contengan *book* pero tambien *bag*

    db.inventory.find(
        { tags: { $all: ["book", "bag"] }},
        { tags:1})

#### size

Que el tamaño del array sea dos elementos:

    db.inventory.find(
        { tags: { $size: 2}},
        { tags:1})


#### elemMatch

Element Match. Usaremso el otro dataset. Donde queremos buscar todas las encuestas, tenga productos *xyz*:

Podriamos hacerlo con *in*

    db.survey.find({
        "results.product": { $in: ["xyz"]}
    })

Y tambienlo con: 

    db.survey.find({
        results: { $elemMatch: { product : "xyz" }}
    })

Tambien podriamos ser mas especificos, e indicarle que tenga un score de 7:

    db.survey.find({
        results: { $elemMatch: { product : "xyz", score: 7 }}
    })

O seguir usando los operadores y decirlo menos o igual que 7

    db.survey.find({
        results: { $elemMatch: { product : "xyz", score: { $lte: 7} }}
    })

![](https://i.imgur.com/lwIgDPa.png)

## Operadores Logicos

